#### Two-Stage Evaluation

1. Objective Issue Detection 

2. Subjective Attribute Evaluation

In [1]:
from utils import collect_ooc_response
collect_ooc_response() 

In [1]:
from src.attribute import Requirement, Attribute

requirements = [
    {
        "name": "Funny",
        "desc": "Assistant should be funny.",
    },
    {
        "name": "Concise",
        "desc": "Response from assistant should be concise and within 2 sentences",
    }
]

requirement = Requirement.make(requirements)

In [2]:
# "data/attribute/*.json" # path to all attributes
from src.attribute import Requirement, Attribute
from src.eval import load_requirements

# Load requirements and create Requirement object
requirements = load_requirements()
requirement = Requirement.make(requirements)


In [6]:
import glob
import json

def load_conversation(folder_dir="data/conversation"):
    """ 
    Load conversation under certain directory
    """
    conversation_files = glob.glob(f"{folder_dir}/*.json")
    for file_path in conversation_files:
        with open(file_path, 'r') as file:
            conversation_data = json.load(file)
            conversations = conversation_data["conversations"]
    return conversations 

In [7]:
load_conversation()

['haha!']

In [2]:
# Now we build the evaluator -- Comparative Score (Base on comparing with the buckets)
from src.eval import AOEval


aoeval = AOEval(requirement) # Async call enabled (Watch-out for request limit)

vllm package not found. HFModel will not be available.
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/fangyuanyu/.cache/huggingface/token
Login successful


In [4]:
import asyncio 

async def run_eval(conversation):
    result = await aoeval.direct_eval(conversation)
    return result

conversation = "haha"
result = await run_eval(conversation)
print(result)

{'Funny': 'Ok', 'Concise': 'Ok'}


In [1]:
from src.detect import run_detection
import json

issue_path = "data/detect/issues.json"
with open(issue_path, 'r') as file:
    detection_issues = json.load(file)
    
query_path = "data/detect/queries.json"
with open(query_path, 'r') as file:
    queries = json.load(file)["queries"]

# run_detection(queries, detection_issues)

vllm package not found. HFModel will not be available.
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/fangyuanyu/.cache/huggingface/token
Login successful


In [6]:
from src.utils import maria_prompt, alex_prompt, Agent, VLLM_MODEL, OpenRouter_Model, FINETUNE_MODEL_NAME, FINETUNE_MODEL_URL, BASE_MODEL_NAME, BASE_MODEL_URL
import os, random
from src.model import get_claude_response
from tqdm import tqdm

# Import llama3 tokenizer
from transformers import AutoTokenizer

# # Import Llama 3 tokenizer
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

# Customer & Agent Prompts
eCoach_prompt = maria_prompt
eAgent_prompt = alex_prompt

# # Model names
# model_names = ["google/gemini-flash-1.5", "openai/gpt-4o", "qwen/qwen-110b-chat", "google/gemini-pro-1.5", "cohere/command-r-plus", "mistralai/mistral-large", "mistralai/mixtral-8x22b-instruct"]
# model_name = random.choice(model_names)

# # Initialize Sales Agent
# sales_model = OpenRouter_Model(model_name)
# sales_agent = Agent(sales_model, tokenizer)  # Assuming tokenizer is not needed for OpenRouter_Model

# # Initialize Customer Agent (Finetuned model)
# customer_model_finetuned = VLLM_MODEL(FINETUNE_MODEL_NAME, FINETUNE_MODEL_URL)
# customer_agent_finetuned = Agent(customer_model_finetuned, tokenizer)  # Assuming tokenizer is not needed for VLLM_MODEL

# # Initialize Customer Agent (Base model)
# customer_model_base = VLLM_MODEL(BASE_MODEL_NAME, BASE_MODEL_URL)
# customer_agent_base = Agent(customer_model_base, tokenizer)  # Assuming tokenizer is not needed for VLLM_MODEL



In [11]:
import json 

# Run Issue Detection with Class Object here 

model_names = ["google/gemini-flash-1.5", "openai/gpt-4o", "qwen/qwen-110b-chat", "google/gemini-pro-1.5", "cohere/command-r-plus", "mistralai/mistral-large", "mistralai/mixtral-8x22b-instruct"]
sales_model_name = random.choice(model_names)

detector = Detector.make(False,
                         sales_model_name,
                         eCoach_prompt, 
                         eAgent_prompt)

issue_history, conversation_history = detector.run()



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
 20%|██        | 2/10 [00:23<01:31, 11.46s/it]

### OOC Detected ###
Issue: out_of_character
Rationale: Alex's response is out of character and doesn't naturally follow from Maria's introduction. Maria introduces herself and mentions hearing about Alex's services, but Alex immediately jumps into discussing a 'set-for-life' product without any proper greeting or acknowledgment of Maria's introduction. The language used by Alex is also unusually cryptic and jargon-filled for an initial meeting, referring to 'concept reboots' and using metaphors that don't align with a typical business conversation. This abrupt and unusual response suggests out-of-character behavior that doesn't match the context of a first-time meeting in a café.
Issue Query: Hi Alex, I'm Maria. And you are Alex, right? I'm glad we could meet here at Café Verde. I've heard good things about your services from my friend, Sam.
Issue Response: Alex: Indeed, Maria, our "set-for-life" is a safeguard. It weaves a safety net, tailored for life's twists. Shall we dive in, or 

 30%|███       | 3/10 [00:33<01:16, 10.99s/it]

### OOC Detected ###
Issue: out_of_character
Rationale: The conversation appears to be out of character because P1's message refers to Alex and asks about a 'concept reboot', suggesting they are discussing a fictional scenario or roleplaying. However, P2's response is written as if it's coming from Alex himself, which is inconsistent with P1's framing. Additionally, the sudden shift to nautical metaphors in P2's response seems out of place given the context of discussing a financial plan.
Issue Query: Sure, Alex. So, shall we dive into the details of the "set-for-life" plan? Or do you think I need a quick concept reboot first?
Issue Response: Alex: Ah, the "set-for-life" plan, your future's anchor. You're thinking security, right? Let's navigate its perks – a smooth sail to financial calm. Ready to set course?
####################


 40%|████      | 4/10 [00:45<01:07, 11.19s/it]

...

### OOC Detected ###
Issue: out_of_character
Rationale: The response from 'p2' (Alex) is out of character for what appears to be a professional insurance discussion. The language used is overly casual, uses unclear metaphors, and doesn't directly address the specific question about premium and coverage adjustments. The response lacks the professionalism and clarity expected in a conversation about insurance plans, especially given the specific nature of the question asked by 'p1'.
Issue Query: Can you also explain how flexible this plan is in terms of adjusting premiums or coverage if my financial situation changes?
Issue Response: Alex: Ah, "set-for-life" insurance, a flexible shield. Life's twists? Covered. Let's play it smart, Maria. Quick check, and we're set. No need to crash and restart here.
####################


 80%|████████  | 8/10 [01:26<00:21, 10.53s/it]

### OOC Detected ###
Issue: out_of_character
Rationale: The response from 'p2' (Alex) appears to be out of character and doesn't directly address the question asked by 'p1'. The response uses vague, poetic language and doesn't provide a clear comparison of the insurance plan to other products in terms of benefits and affordability as requested. It also introduces a name 'Maria' which wasn't mentioned in the original question, suggesting a disconnect in the conversation. The response seems more like a sales pitch than a straightforward answer to the specific question asked.
Issue Query: Thanks for explaining how flexible the plan is. How does it compare to other life insurance products in terms of benefits and affordability?
Issue Response: Alex: "Set-for-life," your life's safety net. Oh, it adapts with ease. Shall we, Maria? A brief check, and you're on track. No crashes, just smooth sailing ahead.
####################


 80%|████████  | 8/10 [01:27<00:21, 10.95s/it]


BadRequestError: Error code: 400 - {'object': 'error', 'message': "This model's maximum context length is 8192 tokens. However, you requested 8925 tokens (8413 in the messages, 512 in the completion). Please reduce the length of the messages or completion.", 'type': 'BadRequestError', 'param': None, 'code': 400}

In [6]:
aoeval.human_annotation("haha", requirement.attributes[1], "Ok")

Attribute: Concise
Conversation:
haha
Suggested bucket: Ok


'Ok'